# Zadatak

Na adresi https://www.foi.unizg.hr/hr/dokumenti nalazi se baza dokumenata Fakulteta organizacije i informatike. Većina dokumenata zadana je u PDF formatu, no neki od dokumenata su čitki (vektorski format) dok su drugi skenirani (rasterski format). Potrebno je:
1) Implementirati program koji će skinuti sve PDF dokumente na lokalno računalo (ili Google Colab / Drive direktorij)
2) Kreirati bazu podataka (SQLite) koja će sadržavati jednu tablicu "dokument". Tablica treba sadržavati šifru dokumenta (autonumber), naslov dokumenta, putanju datoteke (path i filename na lokalnom računalu), URL adresu s koje je dokument skinut, datum, te tekstualni sadržaj dokumenta.
3) Posebno za zadnje polje u tablici potrebno je svaki dokument učitati i ekstrahirati tekst (primjerice putem Python modula PDFMiner - https://github.com/pdfminer/pdfminer.six). Ukoliko je dokument skeniran potrebno je koristiti odgovarajući OCR modul za ekstrakciju teksta (npr. PyTesseract https://pypi.org/project/pytesseract/).
4) Omogućiti pristup skupu podataka putem REST API-ja koji omogućuje isključivo pretraživanje podataka putem:(a) pretraživanja ključnih riječi u tekstu (npr. boolean search), (b) pretraživanje ključnih riječi u naslovu (c) pregled prema datumu (npr. dokumenti od DATUM do DATUM), (d) izlistavanje svih dokumenata.

# Prikupljanje podataka

In [99]:
!mkdir -p data/pdfs

In [98]:
# General
import os
import sys
import csv

# Scraping
import requests
from bs4 import BeautifulSoup

# Data manipulation and processing
import numpy as np
import pandas as pd

In [18]:
url = 'https://www.foi.unizg.hr/hr/dokumenti'

In [19]:
response = requests.get(url)

In [20]:
response

<Response [200]>

In [25]:
soup = BeautifulSoup(response.text, 'html')

In [30]:
print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="ie6 ie" lang="hr" dir="ltr"> <![endif]-->
<!--[if IE 7]>    <html class="ie7 ie" lang="hr" dir="ltr"> <![endif]-->
<!--[if IE 8]>    <html class="ie8 ie" lang="hr" dir="ltr"> <![endif]-->
<!--[if IE 9]>    <html class="ie9 ie" lang="hr" dir="ltr"> <![endif]-->
<!--[if !IE]> -->
<html dir="ltr" lang="hr">
 <!-- <![endif]-->
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="https://www.foi.unizg.hr/sites/default/files/favicon_0_1.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
  <meta content="Drupal 7 (https://www.drupal.org)" name="generator"/>
  <link href="https://www.foi.unizg.hr/hr/dokumenti" rel="canonical"/>
  <link href="https://www.foi.unizg.hr/hr/dokumenti" rel="shortlink"/>
  <meta content="FOI" property="og:site_name"/>
  <meta content="article" property="og:type"/>
  <meta content="https://www.foi.unizg.hr/hr/dokumenti" property="og:url"/>
  <meta content="Dokumenti" propert

## Downloading PDF

In [24]:
def download_pdf(url, path):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Provjerava HTTP greške

        file_name = url.split('/')[-1]
        file_path = os.path.join(path, file_name)

        with open(file_path, 'wb') as f:
            f.write(response.content)

        print(f"Dokument je preuzet i spremljen: {file_path}")
    except requests.exceptions.RequestException as e:
        print(f"Došlo je do greške prilikom preuzimanja: {e}")


## Create a CSV data entry

In [42]:
url = 'https://www.foi.unizg.hr/hr/dokument/izmjenjena-odluka-pok-2023'

In [43]:
response = requests.get(url)
response

<Response [200]>

In [44]:
soup = BeautifulSoup(response.text, 'html.parser')

In [64]:
link = soup.find('a', href=True, type=lambda value: value and 'application/pdf' in value)['href']
datum = soup.find('div', class_ = 'datum').text

In [77]:
fields = soup.find_all('div', class_ = 'field-item even')

for i in range(len(fields)):
    fields[i] = fields[i].text.strip()

In [85]:
ime_datoteke = fields[0]
vrsta_dokumenta = fields[1]
kategorija_dokumenta = fields[2]
datum = datum.strip().split('Kreirano: ')[1]

In [90]:
ime_datoteke, link, datum, vrsta_dokumenta, kategorija_dokumenta

('odluka_o_izmjeni_odluke_pok_2023_10_12.pdf',
 'https://www.foi.unizg.hr/sites/default/files/odluka_o_izmjeni_odluke_pok_2023_10_12_0.pdf',
 '18.12.2023',
 'Odluka',
 'Kvaliteta')

In [94]:
# Definiranje tuple-a
data_tuple = (ime_datoteke, link, 
              datum, vrsta_dokumenta, 
              kategorija_dokumenta)

# Putanja do CSV datoteke koja će se kreirati
csv_file_path = './data/document_data.csv'

# Kreiranje i pisanje u CSV datoteku
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Pisanje headera
    writer.writerow(['Ime datoteke', 'URL', 'Datum kreiranja', 'Vrsta dokumenta', 'Kategorija dokumenta'])
    # Pisanje sadržaja tuple-a
    writer.writerow(data_tuple)
